# 0.0. Imports

In [1]:
import joblib
import pandas as pd
import pandera
from sklearn.model_selection import train_test_split
from pandera import Check, Column, DataFrameSchema
from sklearn.pipeline import Pipeline
from feature_engine.discretisation import EqualFrequencyDiscretiser, EqualWidthDiscretiser
from feature_engine.imputation import MeanMedianImputer
from feature_engine.wrappers import SklearnTransformerWrapper
from sklearn.preprocessing import RobustScaler, StandardScaler
from sklearn.metrics import roc_auc_score
from sklearn.linear_model import LogisticRegression

ModuleNotFoundError: No module named 'pandera'

## 0.1. Utils

# 1.0. Data load

In [19]:
class DataLoad:
    """Class data load"""
    
    def __init__(self) -> None:
        pass 
    
    def load_data(self) -> pd.DataFrame:
        """Funcao vai carregar os dados
        
        return:
            pandas DataFrame"""
            
        loaded_data = pd.read_csv('../data/cardio_train.csv', sep=';')
        return loaded_data

In [20]:
dl_class = DataLoad()

In [21]:
df = dl_class.load_data()

In [22]:
df.head()

,id,age,gender,height,weight,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active,cardio
0,0,18393,2,168,62.0,110,80,1,1,0,0,1,0
1,1,20228,1,156,85.0,140,90,3,1,0,0,1,1
2,2,18857,1,165,64.0,130,70,3,1,0,0,0,1
3,3,17623,2,169,82.0,150,100,1,1,0,0,1,1
4,4,17474,1,156,56.0,100,60,1,1,0,0,0,0


In [ ]:
class DataValidation:
    """"""
    def __init__(self, columns_to_use) -> None:
        self.columns_to_use = columns_to_use
        
    def check_shape_data(self, dataframe: pd.DataFrame) -> bool:
        try:
            print('Validacao iniciou')
            dataframe.columns = self.columns_to_use
            return True 
        except Exception as e:
            print(f'Validacao errou: {e}')
            return False
        
    def check_columns(self, dataframe: pd.DataFrame) -> bool:
        schema = DataFrameSchema(
                {
                    "target": Column(int, Check.isin([0, 1]), Check(lambda x: x > 0), coerce=True),
                    "TaxaDeUtilizacaoDeLinhasNaoGarantidas": Column(float, nullable=True),
                    "Idade": Column(int, nullable=True),
                    "NumeroDeVezes30-59DiasAtrasoNaoPior": Column(int, nullable=True),
                    "TaxaDeEndividamento": Column(float, nullable=True),
                    "RendaMensal": Column(float, nullable=True),
                    "NumeroDeLinhasDeCreditoEEmprestimosAbertos": Column(int, nullable=True),
                    "NumeroDeVezes90DiasAtraso": Column(int, nullable=True),
                    "NumeroDeEmprestimosOuLinhasImobiliarias": Column(int, nullable=True),
                    "NumeroDeVezes60-89DiasAtrasoNaoPior": Column(int, nullable=True),
                    "NumeroDeDependentes": Column(float, nullable=True)
                }
            )
        try:
            schema.validate(dataframe)
            print("Validation columns passed...")
            return True
        except pandera.errors.SchemaErrors as exc:
            print("Validation columns failed...")
            pandera.display(exc.failure_cases)
        return False